In [1]:
import tensorflow as tf
import keras
#import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np
import json
import pandas as pd
from statistics import mean
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam #SGD, Adadelta, RMSprop
from keras import layers
from tensorflow.keras import regularizers
#from sklearn import preprocessing

In [2]:
# Open and Read JSON file's data

file = open(r"shallow-frequencies/shallow-frequencies.json", "r", encoding='utf-8')
js = file.read()
data = json.loads(js)
file.close()

In [3]:
# Get all the proofs
proofs = data["triples"]

types = [row["name"] for row in data["types"]]
terms = [row["name"] for row in data["terms"]]

In [4]:
input_list = []
output_list = []

len_types = len(types)
len_terms = len(terms)

for proof in proofs:
    input_def = [0]*len_types
    output_def = [0]*len_terms
    
    typs = set(proof['types'])
    for typ in typs:
        input_def[types.index(typ)] += proof['types'].count(typ)
    input_list.append(input_def)

    trms = set(proof['terms'])
    for trm in trms:
        output_def[terms.index(trm)] += proof['types'].count(trm)
    output_list.append(output_def)

In [5]:
input_list = np.asarray(input_list)
output_list = np.asarray(output_list)

In [6]:
no_test = 300
x_train, x_test, y_train, y_test = train_test_split(input_list, output_list, test_size=no_test/len(proofs))

In [7]:
model = Sequential()

model.add(Dense(200, input_shape = (len_types,), activation = "softplus"))
model.add(Dropout(0.2))

model.add(Dense(len_terms, activation = "softplus"))
model.add(Dropout(0.05))

model.compile(Adam(learning_rate = 0.003), "categorical_crossentropy", metrics = ["KLDivergence"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               142000    
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 3337)              670737    
                                                                 
 dropout_1 (Dropout)         (None, 3337)              0         
                                                                 
Total params: 812,737
Trainable params: 812,737
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test))

Epoch 1/20
204/204 [==============================] - 3s 9ms/step - loss: 5.0764 - kullback_leibler_divergence: 1.6012 - val_loss: 2.8473 - val_kullback_leibler_divergence: 1.3922
Epoch 2/20
204/204 [==============================] - 2s 8ms/step - loss: 3.5814 - kullback_leibler_divergence: 1.8739 - val_loss: 2.2829 - val_kullback_leibler_divergence: 1.4005
Epoch 3/20
204/204 [==============================] - 2s 8ms/step - loss: 2.9000 - kullback_leibler_divergence: 1.7497 - val_loss: 1.9425 - val_kullback_leibler_divergence: 1.2448
Epoch 4/20
204/204 [==============================] - 2s 8ms/step - loss: 2.6321 - kullback_leibler_divergence: 1.4575 - val_loss: 1.8271 - val_kullback_leibler_divergence: 1.0920
Epoch 5/20
204/204 [==============================] - 2s 8ms/step - loss: 2.4577 - kullback_leibler_divergence: 1.2741 - val_loss: 1.6726 - val_kullback_leibler_divergence: 1.0077
Epoch 6/20
204/204 [==============================] - 2s 8ms/step - loss: 2.2491 - kullback_leibler_

In [12]:
def get_score(y_test, y_pred):
    
    df = pd.DataFrame(list(zip(y_test, y_pred)), columns =['y_test', 'y_pred'])
    df = df.sort_values(by="y_pred", ascending=False)
    
    df['y_pred'] = range(len(df))
    sum_val = df['y_test'].sum()
    
    if sum_val:
        return (df['y_test']*df['y_pred']).sum()/sum_val
    else:
        return 'no elem'

In [13]:
def avg_score(x_tests, y_tests):
    
    y_preds = model.predict(x_tests)
    
    list_score = []
    
    for i in range(len(y_preds)):
        score = get_score(y_tests[i], y_preds[i])
        if score != 'no elem':
            list_score.append(score)
    
    print('\n',f"Average score of {len(list_score)} elements (leaving out empity terms):")
    
    avg_per = mean(list_score)
    print(f" Average rank elements is {round(avg_per)} out of {len(data['terms'])} total terms.")
    
    return avg_per

In [14]:
avg_score(x_test, y_test)

10/10 [==============================] - 0s 3ms/step

 Average score of 88 elements (leaving out empity terms):
 Average rank elements is 55 out of 3337 total terms.


55.27315607372425

### Result of Prediction for each element in test set:

In [15]:
y_pred = model.predict(x_test)
cut_off = 10000

for i in range(len(y_test)):
    df = pd.DataFrame(list(zip(terms, y_test[i], y_pred[i])), columns =['terms', 'y_test_count', 'y_pred_index'])
    df = df.sort_values(by="y_pred_index", ascending=False)

    df['y_pred_index'] = range(len(df))
    sum_val = df['y_test_count'].sum()

    df.index = range(len(df))

    if sum_val:
        avg = (df['y_test_count']*df['y_pred_index']).sum()/(sum_val)
    else:
        avg = 0
        
    if avg <= cut_off:
        print('\n -  -  -  \n')
        if str(avg) == "0.0":
            print('Average', avg)
            display(df[:1])
        elif avg > 0:
            print('Average', avg)
            display(df[:df['y_test_count'].iloc[::-1].ne(0).idxmax()+1])
        else:
            print('Average', avg, '(no elements in defn)')

10/10 [==============================] - 0s 2ms/step

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 1.2


,terms,y_test_count,y_pred_index
0,Int,2,0
1,Eq,1,1
2,OfNat.ofNat,1,2
3,Int.subNatNat,1,3



 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0.5


,terms,y_test_count,y_pred_index
0,True,1,0
1,Exists,1,1



 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 1.3333333333333333


...

,terms,y_test_count,y_pred_index
0,System.FilePath,1,0



 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0.0


,terms,y_test_count,y_pred_index
0,USize,1,0



 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0.0


,terms,y_test_count,y_pred_index
0,Option,1,0
